## Particle Filter


In [ ]:
import cv2
import numpy as np
VFILENAME = "Video3.mp4"

### **Display the Video**

Create a function that reads the video. While the video is opened, we read the frames.

In [ ]:
def get_frames(filename):
    video = cv2.VideoCapture(filename)
    while video.isOpened():
        ret, frame =video.read()
        if ret:
            yield frame 
        else:
            break
    video.release()        
    yield None

We create a fuction to display the particle. First of all, we check if there are any particles to display. Then if so, we are going to iterate over them and draw a circle in correspondence of their pixel coordinates. Next, if we get a location for the particle, we try to track it throughout the video and draw a red circle in correspondece of that location.

In [ ]:
def display(frame, particles, location, NUM_PARTICLES):
    if len(particles)> 0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0),1)
    if len(location) > 0:
        cv2.circle(frame,location,15,(0,0,255),5)
    cv2_imshow(frame)
    #stop the video if pressing the escape button
    if cv2.waitKey(30)==27:
        if cv2.waitKey(0)==27:
            return True 
    return False

### **Intialize the particles**

In order to initialize the particles, we have to estimate the **state of the target**, meaning its **position** and **velocity** within the video. At the beginning of the video, we don’t know that state. All we know is that the position should lie within the frame somewhere, and the velocity could be in any direction but not moving too fast.

In [ ]:
NUM_PARTICLES = 5000
VEL_RANGE = 0.5
frame_height = 720
frame_width = 1280

We start by initializing the number of total particles and the initial velocity range to be a pixel per frame.



In [ ]:
def initialize_particles():
    particles = np.random.rand(NUM_PARTICLES,4)
    particles = particles * np.array((frame_width,frame_height, VEL_RANGE,VEL_RANGE))
    particles[:,2:4] -= VEL_RANGE/2.0
    return particles

The particles are created as an array filled with random numbers with one row per particle and four columns. The fist two columns are the coordinates of the particles, and the last two colums are their velocity. Since the particles have to lay on the frame, the first two columns have values between zero and the frame dimension. The initial velocity is set to 0.5 and it will be centered to zero so the particles have the possibility to move in both directions. Then we are going to decrement that by half the velocity range. So, we are going to shift the velocities down so that everithing is centered in zero.



Let's display the results:


*  Define an empty list for the location of the particles
*  Initialize the particle
*  Display the results




In [ ]:
location =[]
particles = initialize_particles()


for frame in get_frames(VFILENAME):
    if frame is None: break
    terminate = display(frame, particles, location,NUM_PARTICLES)
    if terminate:
        break

cv2.destroyAllWindows()  

### **Moving Particles**

As you can see from the video, the particles are not moving during video playlback even though they have a velocity. We solve this by creating a function **apply_velocity** in which we increment the particle's x and y velocity component.

In [ ]:
def apply_velocity(particles):
    particles[:,0] += particles[:,2]
    particles[:,1] += particles[:,3]

    return particles

Now, we can see the particles are moving according their velocity

In [ ]:
location = []
particles = initialize_particles()


for frame in get_frames(VFILENAME):
    if frame is None: break
    particles = apply_velocity(particles)

    terminate = display(frame, particles, location, NUM_PARTICLES)
    if terminate:
        break

cv2.destroyAllWindows()    

### **Prevent Particles to fall off the edges**

We prevent the particles to fall off the edges by putting some limit on the particles location. To do so, we will loop over the particles and set an upper and lower boundaries on both x and y coordinates.

In [ ]:
def enforce_edges(particles):
    for i in range(NUM_PARTICLES):
        particles[i,0] = max(0,min(frame_width-1, particles[i,0]))
        particles[i,1] = max(0,min(frame_height-1, particles[i,1]))
    return particles

And the result will be displayed with the following:

In [ ]:
location = []
particles = initialize_particles()


for frame in get_frames(VFILENAME):
    if frame is None: break
    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    terminate = display(frame, particles, location,NUM_PARTICLES)
    if terminate:
        break

cv2.destroyAllWindows()  

### **Measure the quality of the particle**

Let's suppose we want to track the elbow of the person, so we have to check that the color of the pixel sitting under each particle and compare it with the target color. 

To do this, we are going to create an array of zeros to store the color differences calling it **errors**. So we iterate over all the particles and we calculate each color difference. The error is calculated as the mean square difference between the two colors.


In [ ]:
TARGET_COLOR = np.array((66,63, 105))

def compute_errors(particles, frame):
    
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y= int(particles[i,1])
        pixel_color = frame[y, x, :]
        errors[i] = np.sum((TARGET_COLOR - pixel_color)**2)
    
    return errors

### **Assign Weights**

The error is used to compute a weight for each particle. When the error is low, we want the weight to be height. This means that a particle is at location where the pixel color is a good match for the target. 

To do this, we are going to the take the highest error and the subtract off the errors array. Futhermore, we want to prevent the particles from piling up along the edge. So the particle on the edge must have a weight equal to zero.

In [ ]:
def compute_weights(errors):
    weights = np.max(errors) - errors
    
    weights[
        (particles[:,0]==0) |
        (particles[:,0]==frame_width-1) |
        (particles[:,1]==0) |
        (particles[:,1]==frame_height-1) ] = 0  
        
    return weights

### **Resample the weights**

If we normalize the weights so that they sum to one, we can use them as a probability distribution over the particles. So, we are going to build another particle array by sampling from the current particles. The ones with high weight will get chosen many times, the ones with low weight may not be chosen at all. To do this, we are going to use the numpy function **choice**:


*   The first argument is the sampling range (NUM_PARTICLES)
*   The second argument is how many samples we have to take (we need as many sample as we have particles)
*   The third argument is the probability distribution



In [ ]:
def resample(particles, weights,NUM_PARTICLES):
    probabilities = weights / np.sum(weights)
    index_numbers = np.random.choice(
        NUM_PARTICLES,
        size=NUM_PARTICLES,
        p=probabilities)
    particles = particles[index_numbers, :]
    
    x = np.mean(particles[:,0])
    y = np.mean(particles[:,1])
    
    return particles, [int(x), int(y)]

The best guess is the mean over the new particles array

In [ ]:
particles = initialize_particles(NUM_PARTICLES,frame_width,frame_height,VEL_RANGE)


for frame in get_frames(VFILENAME):
    if frame is None: break
    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights,NUM_PARTICLES)

    terminate = display(frame, particles, location,NUM_PARTICLES)
    if terminate:
        break

cv2.destroyAllWindows()

However it wasn't quite a pixel on the target. We need to locate the target and keep tracking the target, even if it moves around the frame or the lighting conditions change. The solution for this is to just add some noise. 

In [ ]:
POS_SIGMA = 0.75 # standard deviation on position
VEL_SIGMA = 0.1 # standard deviation on velocity

def apply_noise(particles,POS_SIGMA,VEL_SIGMA,NUM_PARTICLES):
    noise= np.concatenate(
    (
        np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
        np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
        np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1)),
        np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1))
    
    ),
    axis=1)
    
    particles += noise
    return particles

In [ ]:
particles = initialize_particles(NUM_PARTICLES,frame_width,frame_height,VEL_RANGE)


for frame in get_frames(VFILENAME):
    if frame is None: break
    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights,NUM_PARTICLES)
    particles = apply_noise(particles,POS_SIGMA,VEL_SIGMA,NUM_PARTICLES)

    terminate = display(frame, particles, location,NUM_PARTICLES)
    if terminate:
        break

cv2.destroyAllWindows()

From the result. It looks like the particle cloud is distributed along different subject and not drawn in the target. So we have to make the weights more sensitive to color differences. One possible solution is to square the weights.

In [ ]:
def compute_weights(errors):
    weights = np.max(errors) - errors
    
    weights[
        (particles[:,0]==0) |
        (particles[:,0]==frame_width-1) |
        (particles[:,1]==0) |
        (particles[:,1]==frame_height-1) ] = 0  

    weights = weights**2
        
    return weights

In [ ]:
particles = initialize_particles(NUM_PARTICLES,frame_width,frame_height,VEL_RANGE)


for frame in get_frames(VFILENAME):
    if frame is None: break
    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights,NUM_PARTICLES)
    particles = apply_noise(particles,POS_SIGMA,VEL_SIGMA,NUM_PARTICLES)

    terminate = display(frame, particles, location,NUM_PARTICLES)
    if terminate:
        break

cv2.destroyAllWindows()

Now the particles are more attracted to the t-shirt and the bricks. So let's increase the power of the weights to 8.

In [ ]:
def compute_weights(errors):
    weights = np.max(errors) - errors
    
    weights[
        (particles[:,0]==0) |
        (particles[:,0]==frame_width-1) |
        (particles[:,1]==0) |
        (particles[:,1]==frame_height-1) ] = 0  

    weights = weights**8
        
    return weights

In [ ]:
particles = initialize_particles(NUM_PARTICLES,frame_width,frame_height,VEL_RANGE)


for frame in get_frames(VFILENAME):
    if frame is None: break
    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights,NUM_PARTICLES)
    particles = apply_noise(particles,POS_SIGMA,VEL_SIGMA,NUM_PARTICLES)

    terminate = display(frame, particles, location,NUM_PARTICLES)
    if terminate:
        break

cv2.destroyAllWindows()

Now, it is much better, but we have some spreading of particle cloud and it takes long time to go to the target. So, if we raise the power to a higher power, like 16? Try by yourself!